In [1]:
!pip install -q openai langchain huggingface_hub

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 73.6/73.6 kB 6.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 54.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 25.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 90.0/90.0 kB 9.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.1/49.1 kB 4.3 MB/s eta 0:00:00


In [2]:
import os
os.environ['OPENAI_API_KEY'] = ''
os.environ['HUGGINGFACEHUB_API_TOKEN'] = ''

## Plain conditional generation.

### 1. GPT-3

In [3]:
from langchain.llms import OpenAI

llm = OpenAI(model_name = 'text-davinci-003', temperature = 0.9, max_tokens = 256)

ValidationError: ignored

In [ ]:
text = "Why did the chicken cross the road?"

print(llm(text))

KeyboardInterrupt: ignored

### T5-Flan-large

In [4]:
from langchain.llms import HuggingFaceHub

In [5]:
llm_hf = HuggingFaceHub(repo_id = 'google/flan-t5-large',
                        model_kwargs = {"temperature": 0.9})

In [7]:
text = "Why did the chicken cross the road?"

print(llm_hf(text))

to get to the barn


## Prompt templates.

In [8]:
from langchain import PromptTemplate

restaurant_template = """
I want you to act as a naming consultant for new restaurants.
Return a list of restaurant names. Each name should be short, catchy and easy to remember.
It should relate to the type of restaurant you are naming.

What are some good names of a restaurant that is {restaurant_description}"
"""

prompt_template = PromptTemplate(
    input_variables = ["restaurant_description"],
    template = restaurant_template
)

In [9]:
#Example prompt with one input variable.
description = "a Greek place that serves fresh lamb souvlakis and other Greek"
description_01 = "a burger place that is themed with baseball memorabilia"
description_02 = "a cafe that has live hard rock music and memorabilia"

#Sample prompt fot the first description.
prompt_template.format(restaurant_description = description)

'\nI want you to act as a naming consultant for new restaurants.\nReturn a list of restaurant names. Each name should be short, catchy and easy to remember.\nIt should relate to the type of restaurant you are naming.\n\nWhat are some good names of a restaurant that is a Greek place that serves fresh lamb souvlakis and other Greek"\n'

In [19]:
#Query the model with the prompt template.
from langchain.chains import LLMChain

chain = LLMChain(llm = llm_hf, prompt = prompt_template)

#Run the chain only specifying the input variable.
print(chain.run(description))

a gyro


In [11]:
print(chain.run(description_01))

The Baseball Burger


In [12]:
print(chain.run(description_02))

The Rock Cafe


## Prompt templates with `few shot learning`.

In [13]:
from langchain import PromptTemplate, FewShotPromptTemplate

In [14]:
#Create a list of few shot examples.
examples = [
    {"word": "happy", "antonym": "sad"},
    {"word": "tall", "antonym": "short"}

]

example_formatter_template = """
Word : {word}
Antonym: {antonym}\n
"""

#Specify the template to format the examples provided.
example_prompt = PromptTemplate(input_variables = ["word", "antonym"],
                                template = example_formatter_template)

In [15]:
#Create the `FewShotPromptTemplate` object.
few_shot_prompt = FewShotPromptTemplate(examples = examples, example_prompt = example_prompt,
                                        prefix = "Give the antonym of every input",
                                        suffix = "Word: {input}\nAntonym:",
                                        #The input variables are the variables that the overall prompt expects.
                                        input_variables = ["input"],
                                        example_separator = "\n\n")


print(few_shot_prompt.format(input = "big"))

Give the antonym of every input


Word : happy
Antonym: sad




Word : tall
Antonym: short



Word: big
Antonym:


In [16]:
#Prompt model.
chain = LLMChain(llm = llm_hf, prompt = few_shot_prompt)

#Run the chain, only specifying the input variable.
print(chain.run("big"))

small
